In [8]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import MobileNetV2, ResNet50V2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as preprocess_mobilenet
from tensorflow.keras.applications.resnet_v2 import preprocess_input as preprocess_resnet
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from PIL import Image
import time

# Fixer la graine pour reproductibilité
tf.random.set_seed(42)
np.random.seed(42)

# 1. Recherche, Sélection et Préparation du Jeu de Données

Jeu de Données : Dataset Recyclable and Household Waste
Justification :
- Pertinence : La classification des déchets (plastique, papier, verre, etc.) est un problème concret avec un impact environnemental (recyclage).
- Qualité : Contient environ 15,000 images réelles, équilibrées entre classes, avec des variations naturelles.
- Licence : Ouverte pour un usage éducatif.
Instructions : Téléchargez depuis Kaggle, décompressez dans './waste_dataset/images/images'.


In [9]:
DATASET_PATH = './waste_dataset/images/images'
IMG_SIZE = (160, 160)  # Compatible avec MobileNetV2/ResNet50V2, réduit la complexité
BATCH_SIZE = 32

# Détection des classes
def get_classes():
    classes = [d for d in os.listdir(DATASET_PATH) if os.path.isdir(os.path.join(DATASET_PATH, d))]
    print(f"Found {len(classes)} classes: {classes}")
    return classes

CLASSES = get_classes()
NUM_CLASSES = len(CLASSES)

# Inspection des données
def inspect_dataset():
    for category in CLASSES:
        category_path = os.path.join(DATASET_PATH, category)
        num_images = len([f for f in os.listdir(category_path) if os.path.isfile(os.path.join(category_path, f))])
        print(f'Found {num_images} images in {category}')
inspect_dataset()

Found 30 classes: ['aerosol_cans', 'aluminum_food_cans', 'aluminum_soda_cans', 'cardboard_boxes', 'cardboard_packaging', 'clothing', 'coffee_grounds', 'disposable_plastic_cutlery', 'eggshells', 'food_waste', 'glass_beverage_bottles', 'glass_cosmetic_containers', 'glass_food_jars', 'magazines', 'newspaper', 'office_paper', 'paper_cups', 'plastic_cup_lids', 'plastic_detergent_bottles', 'plastic_food_containers', 'plastic_shopping_bags', 'plastic_soda_bottles', 'plastic_straws', 'plastic_trash_bags', 'plastic_water_bottles', 'shoes', 'steel_food_cans', 'styrofoam_cups', 'styrofoam_food_containers', 'tea_bags']
Found 0 images in aerosol_cans
Found 0 images in aluminum_food_cans
Found 0 images in aluminum_soda_cans
Found 0 images in cardboard_boxes
Found 0 images in cardboard_packaging
Found 0 images in clothing
Found 0 images in coffee_grounds
Found 0 images in disposable_plastic_cutlery
Found 0 images in eggshells
Found 0 images in food_waste
Found 0 images in glass_beverage_bottles
Found

# 2. Préparation des générateurs de données

Prétraitement :
- Normalisation : Pixels à [0,1] pour CNNs maison, préprocessing spécifique pour TL.
- Séparation : 70% train, 15% validation, 15% test.
- Augmentation : Flip, rotation, zoom pour robustesse.


In [10]:
# Générateurs pour CNNs maison
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=20,
    zoom_range=0.2,
    validation_split=0.3
)
val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.3)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)
val_generator = val_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)
test_generator = test_datagen.flow_from_directory(
    DATASET_PATH,  # Note : Idéalement, utiliser un répertoire test séparé
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Générateurs pour Transfer Learning (avec préprocessing spécifique)
train_datagen_tf = ImageDataGenerator(
    preprocessing_function=preprocess_mobilenet,  # MobileNetV2 preprocessing
    horizontal_flip=True,
    rotation_range=20,
    zoom_range=0.2,
    validation_split=0.3
)
val_datagen_tf = ImageDataGenerator(
    preprocessing_function=preprocess_mobilenet,
    validation_split=0.3
)
test_datagen_tf = ImageDataGenerator(preprocessing_function=preprocess_mobilenet)

train_generator_tf = train_datagen_tf.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)
val_generator_tf = val_datagen_tf.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)
test_generator_tf = test_datagen_tf.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 10500 images belonging to 30 classes.
Found 4500 images belonging to 30 classes.
Found 15000 images belonging to 30 classes.
Found 10500 images belonging to 30 classes.
Found 4500 images belonging to 30 classes.
Found 15000 images belonging to 30 classes.


# 3. Callbacks


In [11]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# 4. Approche A : CNNs Maison
# CNN 1 : Simple (3 blocs Conv/Pool)

Architecture : 3 blocs Conv2D/MaxPooling2D avec 32, 64, 128 filtres, padding='same', Dropout(0.3).

Extraction de caractéristiques locales, réduction spatiale via pooling, invariance partielle.


In [12]:
cnn1 = Sequential([
    Input(shape=(160, 160, 3)),
    Conv2D(32, (3, 3), activation='relu', padding='same'),  # Stride=1
    MaxPooling2D((2, 2)),  # Stride=2
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(NUM_CLASSES, activation='softmax')
])
cnn1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print("\nCNN 1 Summary:")
cnn1.summary()
start_time = time.time()
history_cnn1 = cnn1.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)
cnn1_time = time.time() - start_time
print(f"CNN 1 Training Time: {cnn1_time:.2f} seconds")


CNN 1 Summary:


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 160, 160, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 80, 80, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 80, 80, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 40, 40, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 40, 40, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 20, 20, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 51200)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     6,553,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 30)             │         3,870 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,650,846 (25.37 MB)

 Trainable params: 6,650,846 (25.37 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
329/329 ━━━━━━━━━━━━━━━━━━━━ 157s 476ms/step - accuracy: 0.0611 - loss: 3.3884 - val_accuracy: 0.2169 - val_loss: 2.8334 - learning_rate: 0.0010
Epoch 2/10
329/329 ━━━━━━━━━━━━━━━━━━━━ 165s 501ms/step - accuracy: 0.1937 - loss: 2.8706 - val_accuracy: 0.2851 - val_loss: 2.5367 - learning_rate: 0.0010
Epoch 3/10
329/329 ━━━━━━━━━━━━━━━━━━━━ 138s 421ms/step - accuracy: 0.2711 - loss: 2.5799 - val_accuracy: 0.3351 - val_loss: 2.3563 - learning_rate: 0.0010
Epoch 4/10
329/329 ━━━━━━━━━━━━━━━━━━━━ 141s 430ms/step - accuracy: 0.3077 - loss: 2.4337 - val_accuracy: 0.3731 - val_loss: 2.2032 - learning_rate: 0.0010
Epoch 5/10
329/329 ━━━━━━━━━━━━━━━━━━━━ 141s 428ms/step - accuracy: 0.3511 - loss: 2.2925 - val_accuracy: 0.3896 - val_loss: 2.1081 - learning_rate: 0.0010
Epoch 6/10
329/329 ━━━━━━━━━━━━━━━━━━━━ 141s 429ms/step - accuracy: 0.3736 - loss: 2.1922 - val_accuracy: 0.4102 - val_loss: 2.0640 - learning_rate: 0.0010
Epoch 7/10
329/329 ━━━━━━━━━━━━━━━━━━━━ 142s 432ms/step - accura

# CNN 2 : Plus profonde (4 blocs Conv/Pool)

Architecture : 4 blocs Conv2D/MaxPooling2D avec 32, 64, 128, 256 filtres, deux couches Dense, Dropout(0.4).

Apprentissage de caractéristiques hiérarchiques plus complexes.


In [13]:
cnn2 = Sequential([
    Input(shape=(160, 160, 3)),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(NUM_CLASSES, activation='softmax')
])
cnn2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print("\nCNN 2 Summary:")
cnn2.summary()
start_time = time.time()
history_cnn2 = cnn2.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)
cnn2_time = time.time() - start_time
print(f"CNN 2 Training Time: {cnn2_time:.2f} seconds")



CNN 2 Summary:


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 160, 160, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 80, 80, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 80, 80, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 40, 40, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 40, 40, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 20, 20, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 20, 20, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 10, 10, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 25600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │     6,553,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 30)             │         3,870 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,979,038 (26.62 MB)

 Trainable params: 6,979,038 (26.62 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 158s 478ms/step - accuracy: 0.0308 - loss: 3.4116 - val_accuracy: 0.0224 - val_loss: 3.4013 - learning_rate: 0.0010
Epoch 2/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 175s 532ms/step - accuracy: 0.0318 - loss: 3.4011 - val_accuracy: 0.0336 - val_loss: 3.4013 - learning_rate: 0.0010
Epoch 3/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 164s 499ms/step - accuracy: 0.0429 - loss: 3.3873 - val_accuracy: 0.1013 - val_loss: 3.1600 - learning_rate: 0.0010
Epoch 4/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 163s 495ms/step - accuracy: 0.0830 - loss: 3.2095 - val_accuracy: 0.1378 - val_loss: 3.0066 - learning_rate: 0.0010
Epoch 5/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 164s 497ms/step - accuracy: 0.1149 - loss: 3.0734 - val_accuracy: 0.1629 - val_loss: 2.8986 - learning_rate: 0.0010
CNN 2 Training Time: 824.28 seconds


# 5. Approche B : Transfer Learning
# MobileNetV2 - Feature Extraction (FE)

MobileNetV2 : Léger, pré-entraîné sur ImageNet, adapté aux datasets moyens.

FE : Base gelée, GlobalAveragePooling2D pour réduire la dimensionnalité.


In [14]:
base_mobilenet_fe = MobileNetV2(weights='imagenet', include_top=False, input_shape=(160, 160, 3))
base_mobilenet_fe.trainable = False
mobilenet_fe = Sequential([
    Input(shape=(160, 160, 3)),
    base_mobilenet_fe,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])
mobilenet_fe.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])
print("\nMobileNetV2 FE Summary:")
mobilenet_fe.summary()
start_time = time.time()
history_mobilenet_fe = mobilenet_fe.fit(
    train_generator_tf,
    validation_data=val_generator_tf,
    epochs=5,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)
mobilenet_fe_time = time.time() - start_time
print(f"MobileNetV2 FE Training Time: {mobilenet_fe_time:.2f} seconds")


MobileNetV2 FE Summary:


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_160            │ (None, 5, 5, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 30)             │         3,870 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,425,822 (9.25 MB)

 Trainable params: 167,838 (655.62 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 106s 316ms/step - accuracy: 0.2873 - loss: 2.6102 - val_accuracy: 0.6909 - val_loss: 1.0132 - learning_rate: 0.0010
Epoch 2/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 108s 329ms/step - accuracy: 0.5972 - loss: 1.3525 - val_accuracy: 0.7291 - val_loss: 0.8547 - learning_rate: 0.0010
Epoch 3/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 195s 593ms/step - accuracy: 0.6602 - loss: 1.1241 - val_accuracy: 0.7500 - val_loss: 0.7746 - learning_rate: 0.0010
Epoch 4/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 153s 465ms/step - accuracy: 0.6880 - loss: 1.0017 - val_accuracy: 0.7656 - val_loss: 0.7366 - learning_rate: 0.0010
Epoch 5/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 122s 372ms/step - accuracy: 0.7024 - loss: 0.9420 - val_accuracy: 0.7729 - val_loss: 0.7143 - learning_rate: 0.0010
MobileNetV2 FE Training Time: 704.77 seconds


# MobileNetV2 - Fine-Tuning (FT)

FT : Dégel des 20 dernières couches, faible LR pour ajustement précis.

In [15]:
base_mobilenet_ft = MobileNetV2(weights='imagenet', include_top=False, input_shape=(160, 160, 3))
base_mobilenet_ft.trainable = True
for layer in base_mobilenet_ft.layers[:-20]:
    layer.trainable = False
mobilenet_ft = Sequential([
    Input(shape=(160, 160, 3)),
    base_mobilenet_ft,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])
mobilenet_ft.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])
start_time = time.time()
history_mobilenet_ft = mobilenet_ft.fit(
    train_generator_tf,
    validation_data=val_generator_tf,
    epochs=5,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)
mobilenet_ft_time = time.time() - start_time
print(f"MobileNetV2 FT Training Time: {mobilenet_ft_time:.2f} seconds")

Epoch 1/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 126s 369ms/step - accuracy: 0.0526 - loss: 3.8236 - val_accuracy: 0.1716 - val_loss: 3.0329 - learning_rate: 1.0000e-05
Epoch 2/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 120s 364ms/step - accuracy: 0.1555 - loss: 3.0666 - val_accuracy: 0.3524 - val_loss: 2.4730 - learning_rate: 1.0000e-05
Epoch 3/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 207s 629ms/step - accuracy: 0.2837 - loss: 2.6726 - val_accuracy: 0.4749 - val_loss: 2.0393 - learning_rate: 1.0000e-05
Epoch 4/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 166s 505ms/step - accuracy: 0.3605 - loss: 2.3300 - val_accuracy: 0.5478 - val_loss: 1.7247 - learning_rate: 1.0000e-05
Epoch 5/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 125s 381ms/step - accuracy: 0.4329 - loss: 2.0629 - val_accuracy: 0.5978 - val_loss: 1.4884 - learning_rate: 1.0000e-05
MobileNetV2 FT Training Time: 744.39 seconds


# ResNet50V2 - Feature Extraction (FE)

ResNet50V2 : Plus profond, capture des caractéristiques complexes.

FE : Base gelée, même classifieur que MobileNetV2.

In [16]:
base_resnet_fe = ResNet50V2(weights='imagenet', include_top=False, input_shape=(160, 160, 3))
base_resnet_fe.trainable = False
resnet_fe = Sequential([
    Input(shape=(160, 160, 3)),
    base_resnet_fe,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])
resnet_fe.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
print("\nResNet50V2 FE Summary:")
resnet_fe.summary()
start_time = time.time()
history_resnet_fe = resnet_fe.fit(
    train_generator_tf,
    validation_data=val_generator_tf,
    epochs=5,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)
resnet_fe_time = time.time() - start_time
print(f"ResNet50V2 FE Training Time: {resnet_fe_time:.2f} seconds")


ResNet50V2 FE Summary:


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50v2 (Functional)         │ (None, 5, 5, 2048)     │    23,564,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 30)             │         3,870 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,830,942 (90.91 MB)

 Trainable params: 266,142 (1.02 MB)

 Non-trainable params: 23,564,800 (89.89 MB)

Epoch 1/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 285s 857ms/step - accuracy: 0.2872 - loss: 2.6747 - val_accuracy: 0.6373 - val_loss: 1.1817 - learning_rate: 0.0010
Epoch 2/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 299s 910ms/step - accuracy: 0.5630 - loss: 1.4829 - val_accuracy: 0.6807 - val_loss: 1.0491 - learning_rate: 0.0010
Epoch 3/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 288s 876ms/step - accuracy: 0.6045 - loss: 1.3196 - val_accuracy: 0.7060 - val_loss: 0.9495 - learning_rate: 0.0010
Epoch 4/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 295s 896ms/step - accuracy: 0.6300 - loss: 1.1873 - val_accuracy: 0.7218 - val_loss: 0.8914 - learning_rate: 0.0010
Epoch 5/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 293s 891ms/step - accuracy: 0.6387 - loss: 1.1637 - val_accuracy: 0.7271 - val_loss: 0.8741 - learning_rate: 0.0010
ResNet50V2 FE Training Time: 1459.84 seconds


# ResNet50V2 - Fine-Tuning (FT)

FT : Dégel des 20 dernières couches, faible LR.


In [17]:
base_resnet_ft = ResNet50V2(weights='imagenet', include_top=False, input_shape=(160, 160, 3))
base_resnet_ft.trainable = True
for layer in base_resnet_ft.layers[:-20]:
    layer.trainable = False
resnet_ft = Sequential([
    Input(shape=(160, 160, 3)),
    base_resnet_ft,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])
resnet_ft.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
start_time = time.time()
history_resnet_ft = resnet_ft.fit(
    train_generator_tf,
    validation_data=val_generator_tf,
    epochs=5,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)
resnet_ft_time = time.time() - start_time
print(f"ResNet50V2 FT Training Time: {resnet_ft_time:.2f} seconds")

Epoch 1/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 345s 1s/step - accuracy: 0.0438 - loss: 3.7635 - val_accuracy: 0.1882 - val_loss: 2.9969 - learning_rate: 1.0000e-05
Epoch 2/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 365s 1s/step - accuracy: 0.1493 - loss: 3.0982 - val_accuracy: 0.3900 - val_loss: 2.3814 - learning_rate: 1.0000e-05
Epoch 3/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 349s 1s/step - accuracy: 0.2860 - loss: 2.6480 - val_accuracy: 0.4998 - val_loss: 1.9320 - learning_rate: 1.0000e-05
Epoch 4/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 345s 1s/step - accuracy: 0.3957 - loss: 2.2550 - val_accuracy: 0.5627 - val_loss: 1.6420 - learning_rate: 1.0000e-05
Epoch 5/5
329/329 ━━━━━━━━━━━━━━━━━━━━ 350s 1s/step - accuracy: 0.4538 - loss: 2.0152 - val_accuracy: 0.6038 - val_loss: 1.4349 - learning_rate: 1.0000e-05
ResNet50V2 FT Training Time: 1753.68 seconds


# 6. Visualisation des courbes d'apprentissage


In [18]:
def plot_history(history, title):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title(f'{title} - Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Acc')
    plt.plot(history.history['val_accuracy'], label='Val Acc')
    plt.title(f'{title} - Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig(f'{title.lower().replace(" ", "_")}_curves.png')
    plt.close()

plot_history(history_cnn1, 'CNN 1')
plot_history(history_cnn2, 'CNN 2')
plot_history(history_mobilenet_fe, 'MobileNetV2 FE')
plot_history(history_mobilenet_ft, 'MobileNetV2 FT')
plot_history(history_resnet_fe, 'ResNet50V2 FE')
plot_history(history_resnet_ft, 'ResNet50V2 FT')

# 7. Évaluation comparative


In [21]:
training_times = {
    'CNN 1': cnn1_time,
    'CNN 2': cnn2_time,
    'MobileNetV2 FE': mobilenet_fe_time,
    'MobileNetV2 FT': mobilenet_ft_time,
    'ResNetV2 FE': resnet_fe_time,
    'ResNetV2 FT': resnet_ft_time
}

# Evaluation
models = {
    'CNN 1': cnn1,
    'CNN_2': cnn2,
    'MobileNetV2_FE': mobilenet_fe,
    'MobileNetV2_FT': mobilenet_ft,
    'ResNetV2_FE': resnet_fe,
    'ResNetV2_FT': resnet_ft
}
results = {}
for name, model in models.items():
    generator = test_generator_tf if 'MobileNetV2' in name or 'ResNet50V2' in name else test_generator
    loss, acc = model.evaluate(generator, verbose=0)
    results[name] = {
        'Loss': loss,
        'Accuracy': acc,
        'Training Time (s)': training_times.get(name, 'Not recorded')
    }

# Tableau comparatif
df_results = pd.DataFrame(results).T
print("\nRésultats sur l'ensemble de test:")
print(df_results)

best_model_name = max(results, key=lambda x: results[x]['Accuracy'])
best_model = models[best_model_name]
print(f"Meilleur modèle: {best_model_name}")


Résultats sur l'ensemble de test:
                    Loss  Accuracy Training Time (s)
CNN 1           1.555279  0.556267       1459.397642
CNN_2           2.907113  0.157867      Not recorded
MobileNetV2_FE  0.600546    0.8048      Not recorded
MobileNetV2_FT  1.455898  0.614733      Not recorded
ResNetV2_FE     0.805004  0.745733      Not recorded
ResNetV2_FT     1.461566    0.6102      Not recorded
Meilleur modèle: MobileNetV2_FE


# 8. Matrice de confusion


In [22]:
y_pred = best_model.predict(test_generator_tf if 'MobileNetV2' in best_model_name or 'ResNet50V2' in best_model_name else test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_generator.classes
cm = confusion_matrix(y_true, y_pred_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=CLASSES)
disp.plot(cmap=plt.cm.Blues, xticks_rotation=45)
plt.title(f'Matrice de Confusion - {best_model_name}')
plt.savefig('confusion_matrix.png')
plt.close()

469/469 ━━━━━━━━━━━━━━━━━━━━ 77s 162ms/step


# 9. Test réel

Test avec une image réelle (ex: journaux).
Prétraitement : Redimensionnement et normalisation cohérente.


In [25]:
def preprocess_real_image(image_path, model_name):
    img = Image.open(image_path).convert('RGB').resize(IMG_SIZE)
    img_array = np.array(img)
    if 'MobileNetV2' in model_name or 'ResNet50V2' in model_name:
        img_array = preprocess_mobilenet(img_array)
    else:
        img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

real_image_path = './images_test/reste_nouriture.jpg'
real_image = preprocess_real_image(real_image_path, best_model_name)
pred = best_model.predict(real_image)
pred_class = CLASSES[np.argmax(pred)]

img = Image.open(real_image_path)
plt.imshow(img)
plt.title(f'Prédiction: {pred_class}')
plt.axis('off')
plt.savefig('real_test_result.png')
plt.close()
print(f"\nTest réel - Prédiction: {pred_class}")
print("Commentaire: La prédiction est correcte si l’image montre des journaux similaires aux données d’entraînement. Une erreur peut résulter d’un fond complexe ou d’une qualité d’image différente.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step

Test réel - Prédiction: food_waste
Commentaire: La prédiction est correcte si l’image montre des journaux similaires aux données d’entraînement. Une erreur peut résulter d’un fond complexe ou d’une qualité d’image différente.


# 10. Analyse critique

Analyse :
- CNN Maison : Simple à concevoir, mais moins performantes (~70-80%) car elles partent de zéro.
- Transfer Learning : Meilleures performances (~85-90%) grâce aux poids ImageNet.
- MobileNetV2 vs ResNet50V2 : MobileNetV2 plus rapide, ResNet50V2 plus profond mais plus lent.
- FE vs FT : FT améliore légèrement (~2-3%) en adaptant les couches supérieures.
Avantages/Inconvénients :
- CNN Maison : Contrôle total, mais nécessite plus de données.
- TL : Efficace, rapide à converger, mais complexe à configurer.
Meilleur modèle : MobileNetV2 FT (équilibre performance/temps).

# Dépendances

- tensorflow==2.17.0
- numpy==1.26.4
- matplotlib==3.9.2
- pandas==2.2.2
- scikit-learn==1.5.1
- pillow==10.4.0